In [17]:
import pandas as pd
import numpy as np
import glob
import matplotlib as mpl
import matplotlib.pyplot as plt 
import os
#import more_itertools as mit

In [18]:

FilePathList = "/Users/timmo/Documents/RAPFLAB/code/MStest" 

FileType="csv"
MassNumbersList=[]
os. chdir(FilePathList)
CurrFile = pd.read_csv("masslist.csv")
for i in (range(len(CurrFile))):
    MassNumbersList.append(CurrFile['mz_list'][i])
print(MassNumbersList)
#MSL=MassNumbersList
#print(MSL)
        
# for i in range(len(MassNumbersList)):
#     print(MassNumbersList[i])
# for entry in range(len(MassNumbersList)):
#     #print(mass)
#     print(MassNumbersList[entry])

[87.1, 129.1, 157.1, 173.1, 175.1, 177.1, 245.1, 247.1, 285.1, 287.1, 315.1, 317.1, 385.1]


In [67]:
####Different way of coding it
FilePath = "/Users/timmo/Documents/RAPFLAB/code/MStest/List" ##currently local only, will change to read from google drive
#OutFolder = "Processed"

FileType="csv"
files = sorted(glob.glob("*"+FileType))
#files.sort(key=os.path.getmtime)
num_files = len(files)
OutFolder = "Processed"
outname = "OOA"
AllFiles = pd.DataFrame()

Selected=[]
ErrorRange = 0.005        
Processed=[]
ProcessedX=[]
ProcessedY=[]
NameList=[]

FileType="csv"
files = sorted(glob.glob("*"+FileType))
#print(MSL)
print(files)
os. chdir(FilePath)
for entry in range(num_files): #file x:
    name = files[entry]
    CurrFile = pd.read_csv(files[entry],skiprows=1)
    #print(CurrFile)
    FullSplit = name.split("_") #gets all parts of the file name
    NameList.append(outname+str(FullSplit[2]))
    for mass in range(len(MassNumbersList)): #goes through each mast in the MassesList
        MassHigh=((MassNumbersList[mass])+ErrorRange)
        MassLow=((MassNumbersList[mass])-ErrorRange)
        fixed=CurrFile.drop(columns="#Point")
        fit=fixed["X(Thompsons)"].between(MassLow, MassHigh, inclusive = True)

        for i in range(len(fit)):
             if fit[i]== True:
                    ProcessedX.append(fixed["X(Thompsons)"][i])
                    ProcessedY.append(fixed["Y(Counts)"][i])

             
        if sum(fit) == False:
                  ProcessedX.append(0)
                  ProcessedY.append(0)
        
        
        
#length=((len(MassNumbersList))
length=(int(len(MassNumbersList)))

for i in range((num_files)):   
    IndFile = pd.DataFrame(columns = ["mz_list",NameList[i]+"_mass",NameList[i]+"_count"])
    IndFile["mz_list"]=MassNumbersList
    IndFile[NameList[i]+"_mass"]=ProcessedX[(i*length):((i+1)*length)]
    IndFile[NameList[i]+"_count"]=ProcessedY[(i*length):((i+1)*length)]
    IndFile.to_csv(OutFolder+"/"+NameList[i]+"_Processed.csv")
           

AllFiles[outname+"mz_list"]=MassNumbersList     
for i in range(num_files):
    AllFiles[outname+"_mass"+NameList[i]]=ProcessedX[(i*length):((i+1)*length)]
    AllFiles[outname+"_count"+NameList[i]]=ProcessedY[(i*length):((i+1)*length)]
    
        
AllFiles.to_csv(OutFolder+"/"+outname+"_AllProcessed.csv")

    

['OOApre_50C_50uM_s2_MS.csv', 'OOApst_50C_15uL_MS.csv']
